# Capstone Project Overview

<p>Welcome to my Capstone Project <b><i>Boosting Email Marketing Campaigns with Machine Learning</i></b>. We will explore how to improve the engagement of email campaigns working two important aspects: Getting insights about content engagement using NLP and choosing the best delivery date and time for campaigns using a supervised learning classifier.</p>

Please, download these two files and put them in the root folder of the Jupyter notebook:

* file 1: https://drive.google.com/file/d/1A783Cu-IxPiKkhjXeMlgwtqZxjMSLrSX/view?usp=sharing
* file 2: https://drive.google.com/file/d/1rH-qfnRB2kJrj7E4fQphLeRuEUUY-Eh0/view?usp=sharing


### How is this Notebook Organized?
These are the sections created to understand to work in the required solution:

* Initialization
* Challenge 1: Getting insights from the Content
    * Data Gathering, cleaning, and feature engineering
    * Using NLP to get insights
    * Visualizing the insights with Tableau
* Challenge 2: Choosing the best date and time to send the campaign
    * Data Gathering and cleaning
    * Modeling the solution and feature engineering
    * Choosing the right classifier to solve the problem



## Initialization

In [2]:
#Data structure objects
import pandas as pd
import numpy as np

#Plotting objects
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Data splitting, scaling, transformation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

#Regressions and classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#Evaluation
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#Other
import warnings
import time
import math

warnings.filterwarnings('ignore')

#To use NLP
import os
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize

#This installs "Punkt Sentence Tokenizer"
nltk.download('punkt')



[nltk_data] Downloading package punkt to /Users/sebas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Challenge 1: Getting insights from the Content




### Data Gathering, Feature Engineering

<p>As we will discover which keywords are important to our audience, we need to gather that information from the existent campaigns. In this case, we will analyze 185 campaigns and the engagement to understand which subjects could be essential to generate engaging content.</p>

In [5]:
df = pd.read_csv('20190616-Campaigns.csv',sep=';')
df.index = range(1,len(df)+1)
df['total_engagement'] = df['opens']+df['clicks']
df.head()

,campaignId,subj,opens,clicks,keywords,total_engagement
1,10112603,Que paso en octubre en Simplificacion?,203,18,Productive Simplification Department Newsletter,221
2,10112594,Que paso en octubre en Industria?,34,9,Industry Department Newsletter,43
3,10112583,Que paso en octubre en Comercio?,193,75,Commerce Department Newsletter,268
4,10111394,Obtene un credito para tu PyME con el aval de ...,7319,572,SGR Program Credit Benefit,7891
5,10111394,Obtene un credito para tu PyME con el aval de ...,3,0,SGR Program Credit Benefit,3


In [6]:
len(df)

185

In [8]:
# no nans in this dataframe
df.isnull().values.any()


False

The feature total_engagement will be used as a measure to weight the individual keywords. This weight will be visually compared to the weight represented by the number of times that we use that word in different campaigns.

In [7]:
df['total_engagement'] = df['total_engagement'].astype(int, copy=False) 

### Using NLP to get insights

<p>Now we need to use the NLP tokenize to create the keywords and count the number of times the keyword is used in our campaigns</p>

In [9]:
kWords = ''

for i,r in df.iterrows():

    kWords = kWords + r['keywords'] + ' '
    
print(kWords)

tk = word_tokenize(kWords)
print (tk)


Productive Simplification Department Newsletter Industry Department Newsletter Commerce Department Newsletter SGR Program Credit Benefit SGR Program Credit Benefit Productive Simplification Department Newsletter SMB Program Benefits Simple Export Program Benefits  Event invitation Export Rio Cuarto Cordoba Commerce Department Benefits Event invitation Export Rio Cuarto Cordoba Export Benefit Program Education SMB Benefit Credit SGR SMB Program Register SMB Benefits Certificate Tools SMB Opportunities Growth Digital Sign Delivery Export New Markets Export Cost Reduction Benefits Simplification Competitivity International trade Harbour Registry National Harbour Metrobus Network Growth Metrobus Neighborhood Buenos Aires Program Event Design Argentina Credit SGR SMB Program Register SMB Benefits Certificate Training Program Discounts Event Export Coast Newsletter International Trade Event Google Salta Program Register SMB Benefits Certificate Training Newsletter Program CTIT Newsletter Pro

In [11]:
from nltk.probability import FreqDist
fdist = FreqDist()

for word in tk:
    fdist[word.lower()]+=1

fdist

FreqDist({'program': 59, 'smb': 48, 'event': 37, 'benefits': 35, 'export': 34, 'department': 29, 'newsletter': 29, 'argentina': 27, 'news': 20, 'credit': 17, ...})

Finally, we create the dataframe to contain both the number of times the word appears in the campaign and the engagement obtained.

In [12]:
dfKeywords = pd.DataFrame(pd.Series(dict(fdist)))
dfKeywords.reset_index(inplace=True)
dfKeywords.columns=['keyword','occurrences']

dfKeywords.head()

,keyword,occurrences
0,productive,10
1,simplification,11
2,department,29
3,newsletter,29
4,industry,6


In [13]:
#here we tokenize the keywords of each campaign in the original dataframe
df['tokens']=''

for i,r in df.iterrows():

    df.at[i,'tokens']=word_tokenize(r['keywords'].lower())

df.head()

,campaignId,subj,opens,clicks,keywords,total_engagement,tokens
1,10112603,Que paso en octubre en Simplificacion?,203,18,Productive Simplification Department Newsletter,221,"[productive, simplification, department, newsl..."
2,10112594,Que paso en octubre en Industria?,34,9,Industry Department Newsletter,43,"[industry, department, newsletter]"
3,10112583,Que paso en octubre en Comercio?,193,75,Commerce Department Newsletter,268,"[commerce, department, newsletter]"
4,10111394,Obtene un credito para tu PyME con el aval de ...,7319,572,SGR Program Credit Benefit,7891,"[sgr, program, credit, benefit]"
5,10111394,Obtene un credito para tu PyME con el aval de ...,3,0,SGR Program Credit Benefit,3,"[sgr, program, credit, benefit]"


To finish, we need to put the engagement score in each Keyword along with the number of times it appears in the campaigns.

In [14]:
dfKeywords['engagement']=0
for i,r in df.iterrows():
    for li in r['tokens']:
        indx = dfKeywords.index[(dfKeywords['keyword']==li)]
        keyword_engagement = dfKeywords['engagement'][(dfKeywords['keyword']==li)]
        campaigne_engagement = r['total_engagement']
        dfKeywords.at[indx,'engagement']=keyword_engagement + campaigne_engagement

dfKeywords.head()

,keyword,occurrences,engagement
0,productive,10,7072
1,simplification,11,9946
2,department,29,43192
3,newsletter,29,19560
4,industry,6,1605


In [15]:
#save the file to use it with Tableau
dfKeywords.to_csv('20190611_keywords_score_output.csv')

### Visualizing the insights with Tableau

<p>In the report generated in Tableau, we can see many opportunities to deliver engaging content. For example, we can create content more related to credits, expiration policy of certificates, registration to programs, among others. We can also note that we are talking too much about international trade and people is not engaged with the subject. This kind of ideas are valuable tools to plan the content marketing of the company and this is an easy way to know where to go in a glimpse.</p>

<img src="https://i.postimg.cc/5ypjSTmg/NLP-Tableau.png" alt="Tableau">



## Challenge 2: Choosing the best date and time to send the campaign (*)



<p>Reaching out the people at the right moment it's probably one of the most important factors involved in engaging the audience. Nowadays, there are too many emails waiting for the user on inbox and chances are we will increase the open rate if we send our email when the user is online. How is it possible to increase the probability of reach the audience at the best date and time of the week?</p>

<p>We need to train a classifier model of Machine Learning using all the relevant features available and decide the appropriate moment for each company.</p>

### The Data

<p>In this case, the nonprofit entity has many departments holding information about the companies enrolled in different programs and benefits. All the details about the companies are stored in the State database and, all the information on the email campaigns are stored in another place. </p>

<img src="https://i.postimg.cc/D08Lm1sB/model.png" alt="Model">



#### <p>* IMPORTANT NOTE: This notebook summarizes the best outcome after a number of different tries. If you want to check all the tests in code and the result for each test, please, go to the DRAFT notebook provided as a separate file.</p>

### Data Gathering and Cleaning

<p>In this step, we obtained the relevant data from email campaigns and the particularities of each sending. Also, all the information related to the company that could be relevant such as:</p>

<p>Important feature descriptions for email campaigns:</p>

* 'campaignId': is the ID of the campaign.
* 'subscriberEmail': is the subscriber email, in this case, is encrypted.
* 'cuit': is the company ID given by the government. In Canada is also known as Business Number.
* 'deliveryStatus': we have interested only the unbounced emails.
* 'lastOpenDate':  when this field is not NAN means the mail was open at this date and time.

Note about data of email campaigns: The data was extracted from the email marketing system database. The original input came on CSV files. To protect privacy, the subscriberEmail was encrypted.

<p>Important feature descriptions for companies:</p>


* cuit: is the company ID given by the government. In Canada is also known as Business Number.
* province: is the province where the company is located
* lat: geographic latitude  
* longit: geographic longitude
* is_employer: 0 or 1 describing if the company has employees.
* size: if the size of the company. Could be MICRO, MEDIUM or LARGE.
* macro_sector: is the macro sector of the economy in which the company fits. 
* fast_growth: 0 or 1 describing if the company is considered as fast growth.
* importing: 0 or 1 describing if the company imports.
* exporting: 0 or 1 describing if the company exports.  
* is_client: : 0 or 1 describing if the company is existent client.

Note about data of email campaigns: The data was extracted from the State database, pre-processed in SQL and extracted to a CSV. The SQL file with the query is data_empresas_target.sql

Let's work getting the information from the files and cleaning all the necesary characteristics of the dataset 


In [16]:
#original columns
source_columns = ['campaignId', 'scheduleddate', 'campaignName','campaignSubject', 'subscriberEmail', 'cuit', 'deliveryStatus','lastOpenDate', 'opensCount', 'clicksCount']
#columns used for the first cleaning
cleaning_columns = ['campaignId', 'scheduleddate', 'campaignName',
       'campaignSubject', 'subscriberEmail', 'cuit', 'deliveryStatus',
       'lastOpenDate', 'opensCount', 'clicksCount']


def valid_file(df,cols):
    return(len(set(cols).intersection(df.columns))<=len(df.columns))

def append_data(dfappend, csv_file,source_columns,sep=''):
    df_sendings_source = pd.DataFrame()
    
    if(sep==''):
        df_sendings_source = pd.read_csv(csv_file)
    else:
        df_sendings_source = pd.read_csv(csv_file, sep)            
    df_sendings_source = df_sendings_source.reset_index(drop=True)
    dftmp = df_sendings_source[source_columns]
    dfappend = dfappend.append(dftmp, ignore_index=True)
    return dfappend

#Debug toolkit
def list_all_columns(df):
    return df.columns.tolist()

#to balance the classes
def lr_class_balance_status(df,y_field):
    positive_observations = len(df[df[y_field]==1])
    negative_observations = len(df[df[y_field]==0])
    all_observations = len(df)
    print(f'Total number of observations: {all_observations}')
    print(f'Positive number of observations: {positive_observations}')
    print(f'Positive number of observations: {negative_observations}')
    if(positive_observations>negative_observations):
        print(f'Imbalance on more Positive number of observations: {(positive_observations/all_observations)*100}% over 100%') 
    else:
        print(f'Imbalance on more Negative number of observations: {(negative_observations/all_observations)*100}% over 100%')         
        


The following code is commented because I used it to encrypt sensitive information before the process.

In [44]:
# df_sendings_cleaning = pd.DataFrame(columns=cleaning_columns)

# df_sendings_cleaning = append_data(df_sendings_cleaning, '201810_reportes_camp.csv',source_columns)
# df_sendings_cleaning = append_data(df_sendings_cleaning, '201811_reportes_camp.csv',source_columns)
# df_sendings_cleaning = append_data(df_sendings_cleaning, '201812_reportes_camp.csv',source_columns, sep=';')
# df_sendings_cleaning = append_data(df_sendings_cleaning, '201901_reportes_camp.csv',source_columns)
# df_sendings_cleaning = append_data(df_sendings_cleaning, '201902_reportes_camp.csv',source_columns)

# from Crypto.Cipher import XOR
# import base64

# def encrypt(key, plaintext):
#     cipher = XOR.new(key)
#     return base64.b64encode(cipher.encrypt(plaintext))


# df_sendings_cleaning['subscriberEmail'] = df_sendings_cleaning['subscriberEmail'].apply(lambda x: encrypt('?????????',x))

# df_sendings_cleaning.to_csv('reportes_camp_encrypted.csv', index=False)


In [46]:
df_sendings_cleaning = pd.read_csv("reportes_camp_encrypted.csv")


df_sendings_cleaning.head()


,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount
0,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'ERESCRgcMhQCDhcGFAwbCx5PFBwVQRMW',2.017256e+10,opened,2018-11-02T13:54,3,0
1,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'FRMaEBUdGwUeFhwBGi8VCREIH10UAB8=',2.032532e+10,opened,2018-11-02T13:45,5,1
2,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'Aw4QHB8KGwoVAhgWNwgfBRkNXRAYAg==',NaN,opened,2018-11-02T13:26,1,0
3,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'EQYSHhUGAAUDEhpdBx0dAAUCEBoYATIDHQAaH1kMHQk=',2.335728e+10,opened,2018-11-02T13:20,3,0
4,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'HQ0cAxIVHw0CAAYzEAITDRxPEBwa',2.031195e+10,opened,2018-11-02T12:13,1,0


In [47]:
df_sendings_cleaning.shape

(2119479, 10)

In [48]:
#drop company id with nulls
df_sendings_cleaning.dropna(subset=['cuit'], inplace=True)
df_sendings_cleaning.shape

(1990886, 10)

In [49]:
#remove duplicates
df_sendings_cleaning.drop_duplicates(inplace=True)
df_sendings_cleaning.shape

(1887933, 10)

In [50]:
#transform the company id from number to string and remove the decimal part
df_sendings_cleaning['cuit'] = df_sendings_cleaning['cuit'].astype(str, copy=False)
df_sendings_cleaning['cuit'] = df_sendings_cleaning["cuit"].str.replace(".0","") 
df_sendings_cleaning.head(5)

,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount
0,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'ERESCRgcMhQCDhcGFAwbCx5PFBwVQRMW',172557229,opened,2018-11-02T13:54,3,0
1,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'FRMaEBUdGwUeFhwBGi8VCREIH10UAB8=',325319535,opened,2018-11-02T13:45,5,1
3,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'EQYSHhUGAAUDEhpdBx0dAAUCEBoYATIDHQAaH1kMHQk=',23357282179,opened,2018-11-02T13:20,3,0
4,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'HQ0cAxIVHw0CAAYzEAITDRxPEBwa',3119484,opened,2018-11-02T12:13,1,0
5,10112603,2018-10-30T20:46,20181030-Newsletter SSP,¿Qué pasó en octubre en Simplificación?,b'HQAHGhQOGxYfIRQeFgYeShMOHg==',3965223,opened,2018-11-02T11:50,1,0


In [52]:
df_sendings_cleaning.drop(df_sendings_cleaning[df_sendings_cleaning['cuit']==''].index, inplace=True)
df_sendings_cleaning.shape

(1887624, 10)

In [53]:
#remove duplicates
df_sendings_cleaning.drop_duplicates(inplace=True)
df_sendings_cleaning.shape

(1887624, 10)

In [54]:
#drop hard and soft bounced
df_sendings_cleaning.drop(df_sendings_cleaning[df_sendings_cleaning['deliveryStatus']=='hardBounced'].index, inplace=True)
df_sendings_cleaning.drop(df_sendings_cleaning[df_sendings_cleaning['deliveryStatus']=='softBounced'].index, inplace=True)

df_sendings_cleaning.shape

(1747005, 10)

In [55]:
#COMPANIES TO JOIN
#load target companies
df_companies_cleaning = pd.read_csv("201903_target_companies.csv")
df_companies_cleaning.head()

,cuit,province,lat,longit,is_employer,size,macro_sector,fast_growth,importing,exporting,is_client
0,20001082001,Buenos Aires,-34.613152,-58.377232,1,MICRO,Comercio,0,0,0,0
1,20001161408,Buenos Aires,-34.613152,-58.377232,1,MICRO,Agro,0,0,0,0
2,20002554292,Buenos Aires,-34.613152,-58.377232,1,MICRO,Comercio,0,0,0,0
3,20002578051,Buenos Aires,-34.613152,-58.377232,1,MICRO,Agro,0,0,0,0
4,20004240732,Buenos Aires,-34.613152,-58.377232,1,MICRO,Comercio,0,1,0,1


In [56]:
#force company id column data type
df_sendings_cleaning['cuit'] = df_sendings_cleaning['cuit'].astype(str, copy=False)
df_companies_cleaning['cuit'] = df_companies_cleaning['cuit'].astype(str, copy=False)

dfsendings = df_sendings_cleaning.merge(df_companies_cleaning, on='cuit', how='inner')
dfsendings.head()

,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount,province,lat,longit,is_employer,size,macro_sector,fast_growth,importing,exporting,is_client
0,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,Tierra del Fuego,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0
1,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,Capacitá al personal de tu PyME,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,Tierra del Fuego,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0
2,10089311,2018-10-16T20:30,20181016-REGISTRO PYME,Obtené tu certificado PyME y accedé a estos be...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,Tierra del Fuego,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0
3,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,Tierra del Fuego,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0
4,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,Tierra del Fuego,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0


In [57]:
dfsendings.shape

(47443, 20)

### Modeling the solution and feature engineering


<p>In order to be able to classify the best delivery date and time, it is necessary to analyze the deliveries with open emails as a result. We will select the messages sent to all the companies that have all the attributes that we will use in the classification and in turn, have had open the mail.</p>

<p>To classify the specific day and time of the week on which the campaign should be sent, it is necessary to conceptualize the week in intervals.</p>

<p>During the week, from Monday to Friday the morning and the afternoon are recognized, each being half a day. Saturday and Sunday have no division between morning and afternoon. That should be 12 intervals per week to classify.</p>

<p>Before trying this division, we tried to divide the day into six sections of 4 hours for greater accuracy, but the result was not right in terms of classification. In this case, we had 42 intervals per week.</p>

####  Dummies

<p> It's necessary to create dummy variables for macro_sector, province, and every DateTime, interval.</p>


In [58]:

#create the dummy engaged
dfsendings['engaged']=0

#update the dummies with the correct value
for i,r in dfsendings.iterrows():
    if(not (pd.isna(dfsendings.iloc[i]['lastOpenDate']))):
        dfsendings.at[i,'engaged']=1

dfsendings.head()

,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount,...,lat,longit,is_employer,size,macro_sector,fast_growth,importing,exporting,is_client,engaged
0,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0,1
1,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,Capacitá al personal de tu PyME,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0,0
2,10089311,2018-10-16T20:30,20181016-REGISTRO PYME,Obtené tu certificado PyME y accedé a estos be...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0,0
3,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0,1
4,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,-53.771115,-67.726303,1,MEDIANA,Servicios,0,0,0,0,0


In [61]:
def nameOfOpenDateTimeBinDummyWider(objConvert):
    vDateTime = pd.to_datetime(objConvert)
    vDay_name = vDateTime.day_name()
    if(vDay_name =='Saturday'):
        nameOfDummy = 'wo_'+vDay_name
    elif(vDay_name =='Sunday'):
        nameOfDummy = 'wo_'+vDay_name
    else:
        try:
            nameOfDummy = 'wo_'+vDay_name+'_'+t_binWider(vDateTime.hour)
        except:
            print('objConvert: ',objConvert)
            print('vDay_name: ',vDay_name)
            print('vDateTime.hour: ',vDateTime.hour)
            raise Exception
            
    return nameOfDummy

def t_binWider(t):
    return {
        0:'morning',
        1:'morning',
        2:'morning',
        3:'morning',
        4:'morning',
        5:'morning',
        6:'morning',
        7:'morning',
        8:'morning',
        9:'morning',
        10:'morning',
        11:'morning',
        12:'morning',
        13:'afternoon',
        14:'afternoon',
        15:'afternoon',
        16:'afternoon',
        17:'afternoon',
        18:'afternoon',
        19:'afternoon',
        20:'morning',
        21:'morning',
        22:'morning',
        23:'morning'}[t]


timeBinsWider = ['morning','afternoon']
dayBins = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [62]:
#generate column for sednings, initialize all with 0
for d in dayBins:
    for t in timeBinsWider:
        if(d =='Saturday'):
            dfsendings['wo_'+d] = 0
        elif(d =='Sunday'):
            dfsendings['wo_'+d] = 0
        else:
            dfsendings['wo_'+d +'_'+t] = 0

dfsendings.head()

,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount,...,wo_Tuesday_morning,wo_Tuesday_afternoon,wo_Wednesday_morning,wo_Wednesday_afternoon,wo_Thursday_morning,wo_Thursday_afternoon,wo_Friday_morning,wo_Friday_afternoon,wo_Saturday,wo_Sunday
0,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,0,0,0,0,0,0,0,0,0,0
1,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,Capacitá al personal de tu PyME,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10089311,2018-10-16T20:30,20181016-REGISTRO PYME,Obtené tu certificado PyME y accedé a estos be...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,0,0,0,0,0,0,0,0,0,0
4,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
#update the dummies with the correct value
for i,r in dfsendings.iterrows():
    openDateTime = r['lastOpenDate']
    try:
        if(not pd.isna(openDateTime)):
            dfsendings.at[i,nameOfOpenDateTimeBinDummyWider(openDateTime)]=1
    except:
        print("pd.isna(dfsendings.iloc[i]['lastOpenDate']: ", pd.isna(dfsendings.iloc[i]['lastOpenDate']))
        print("r['lastOpenDate']:  ", r['lastOpenDate'])
        print("r:  ", r)
        break
        
dfsendings.head()

,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount,...,wo_Tuesday_morning,wo_Tuesday_afternoon,wo_Wednesday_morning,wo_Wednesday_afternoon,wo_Thursday_morning,wo_Thursday_afternoon,wo_Friday_morning,wo_Friday_afternoon,wo_Saturday,wo_Sunday
0,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,0,0,0,0,0,0,1,0,0,0
1,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,Capacitá al personal de tu PyME,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10089311,2018-10-16T20:30,20181016-REGISTRO PYME,Obtené tu certificado PyME y accedé a estos be...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,0,0,0,0,0,0,1,0,0,0
4,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,notOpened,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
#create more dummies from other values: province and macro_sector
dfsendings.index = range(1,len(dfsendings)+1)

dums = pd.get_dummies(dfsendings[['province', 'macro_sector']])
dfsendings = pd.concat([dfsendings,dums],axis=1)

To manage every group of dummies, we create lists with the name of the fields.

In [65]:
openingDateTimeDummyListWider = list(filter(lambda s: s.startswith('wo_'), dfsendings.columns))
provinceDummyList = list(filter(lambda s: s.startswith('province_'), dfsendings.columns))
otherDummyList = ['fast_growth','importing','exporting','is_client']
macroSectorDummyList = list(filter(lambda s: s.startswith('macro_sector_'), dfsendings.columns))
#y_field = ''

As we need to use only engaged results to train our models, we need to filter the dataframe

In [66]:
#we take only the group with open emails
dfengagedSendigns = dfsendings[dfsendings['engaged']==1]
len(dfengagedSendigns)


16425

Now we must create the column with the label to be predicted

In [68]:
#create one column with the new Y containing the name of the datetime bin
dfengagedSendigns['predicted_dt']=''

#update the dummies with the correct value
for i,r in dfengagedSendigns.iterrows():
    done = False
    iColumn=0
    while(not done):
#            if(dfengagedSendigns.at[i,openingDateTimeDummyList[iColumn]]==1):
        if(r[openingDateTimeDummyListWider[iColumn]]==1):
            dfengagedSendigns.at[i,'predicted_dt']=openingDateTimeDummyListWider[iColumn]
            done = True
        else:
            iColumn+=1

In [69]:
dfengagedSendigns.head()

,campaignId,scheduleddate,campaignName,campaignSubject,subscriberEmail,cuit,deliveryStatus,lastOpenDate,opensCount,clicksCount,...,province_Santiago del Estero,province_Tierra del Fuego,province_Tucuman,macro_sector_Agro,macro_sector_Comercio,macro_sector_Construccion,macro_sector_Industria,macro_sector_Mineria,macro_sector_Servicios,predicted_dt
1,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,0,1,0,0,0,0,0,0,1,wo_Friday_morning
4,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,NaN,b'FwQBFhkMGwUwFQEWGQsXCBYIHRcSAx8RHgUcXRQAH0oR...,33654196589,opened,2018-11-02T03:22,2,0,...,0,1,0,0,0,0,0,0,1,wo_Friday_morning
9,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'AwAdCTcGARQdTxYXAkETFg==',33641241739,opened,2018-11-01T22:27,9,0,...,0,0,0,0,0,0,0,0,1,wo_Thursday_morning
10,10111394,2018-10-30T14:27,20181016-SGR GRUPO B,Obtené un crédito para tu PyME con el aval de ...,b'GQ8VHDcGARQdTxYXAkETFg==',33641241739,opened,2018-10-30T18:01,1,0,...,0,0,0,0,0,0,0,0,1,wo_Tuesday_afternoon
11,10100120,2018-10-24T20:35,20181023-Credito fiscal GRUPO A,Capacitá al personal de tu PyME,b'GQ8VHDcGARQdTxYXAkETFg==',33641241739,opened,2018-10-25T12:30,4,1,...,0,0,0,0,0,0,0,0,1,wo_Thursday_morning


### Choosing the right classifier to solve the problem

<p>Prior to getting the right classifier, we tried the following classifiers in a loop with different hyperparameters and regularizations:</p>

* Logistic Regression
* KNN
* Decision Trees
* Support Vector Machine Classifier

<p>Finally, and after trying <b>Support Vector Machine Classifier</b> also with different hyperparameters, the best configuration for the whole model was the following.</p>

<br><br>


In [70]:
from sklearn.svm import SVC
my_kernel_SVM = SVC()
#my_linear_kernel_SVM = SVC(kernel='linear')

X= dfengagedSendigns[provinceDummyList+macroSectorDummyList+otherDummyList]
y= dfengagedSendigns['predicted_dt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)


#Scale and Transform data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



#Fit the data
my_kernel_SVM.fit(X_train,y_train)
#Get the score
print("TRAIN: Kernel SVM score: %3.3f" % (100*my_kernel_SVM.score(X_train,y_train)))
print("TEST: Kernel SVM score: %3.3f" % (100*my_kernel_SVM.score(X_test,y_test)))


TRAIN: Kernel SVM score: 18.887
TEST: Kernel SVM score: 15.342


#### What does it mean?

<p>Let's analyze what we did and if it's ok, besides what represents the result.</p>

<p>To start, we should say that the probability of getting the right date and time is 1/12 = 0,0833</p>
<p>In the strange case that we wouldn't apply any filter of size, macro_sector or province for example just with SVC we would have 0,1534.</p>
<p>This is 1.84 more chances for that mail to be sent in the right moment and increase the possibilities for opening</p>

<p>But this is not the most common scenario. The typical scenario is at least filter two variables as Province and Macro Sector.</p>

<p>Let's see what happen in this case with an example. In the next example, we can see the chances of increment six times the probability of sending the campaign at the right moment. </p>
<br>

In [71]:
dfsendings_filtered = dfengagedSendigns[(dfengagedSendigns['province']=='Salta') & (dfengagedSendigns['macro_sector']=='Industria')]


from sklearn.svm import SVC
my_kernel_SVM = SVC()
#my_linear_kernel_SVM = SVC(kernel='linear')

X= dfsendings_filtered[macroSectorDummyList+otherDummyList]
y= dfsendings_filtered['predicted_dt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)


#Scale and Transform data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



#Fit the data
my_kernel_SVM.fit(X_train,y_train)
predictions = my_kernel_SVM.predict(X_test) 
#Get the score
print("TRAIN: Kernel SVM score: %3.3f" % (100*my_kernel_SVM.score(X_train,y_train)))
print("TEST: Kernel SVM score: %3.3f" % (100*my_kernel_SVM.score(X_test,y_test)))

TRAIN: Kernel SVM score: 57.143
TEST: Kernel SVM score: 50.000
